# MEV Attacks

## Imports

In [1]:
import pandas as pd
import plotly.io as pio
from sqlalchemy import create_engine

from mev_attacks import utils
import requests

pd.options.plotting.backend = "plotly"
pio.templates.default = "none"

In [4]:
DB_AVAIL = False
if DB_AVAIL:
    user = "postgres"
    password="password"
    host = "localhost"
    engine   = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}/mev_inspect', pool_recycle=3600)
    conn = engine.connect();

## Sandwich Attack

### Description

A sandwich attack involves executing transactions immediately before and immediately after a swap transaction on a decentralized exchange that sets prices according to a known formula. Many of the largest decentralized exchanges utilize a constant product formula such that the product of the supply of each token in a pool must remain constant.

For example, if there were a pool of a pair of tokens A and B each with an initial supply of 10, the product would need to remain constant at 100. To determine the number of B tokens that would be received for 1 A token, you would take 100 divided by 11, the new supply of A, to determine that after the transaction there needs to be 9.090909 B tokens in order to keep the product of the supplies constant at 100. Therefore, you could swap 1 A token for 0.90909 B tokens.

|        | A         | B       | k / price    |
| ------ | -------: | -------: |------------: |
| $t_0$  | 10.0000  | 10.0000  |    100.0000  |
| $user$ |  1.0000  | -0.9091  |      0.9091  |
| $t_3$  | 11.0000  |  9.0909  |    100.0000  |

Since transactions are written together in blocks, the final amount received for an individual transaction depends on the other transactions in the block that get executed before it. In order to protect users from unlimited exposure to changes in their received price, one of the parameters they specify is the amount of slippage they will suffer before their transaction will simply not be executed.

The sandwich attack will execute transactions on either side of a swap transaction to capture the maximum allowable slippage. The default slippage for Uniswap is 0.5%, but it can be set to anything. To illustrate the nature of the attack, assume that the allowable slippage in the above transaction was specified to be 10%, meaning that as few as 0.8181 B tokens could be received for 1 A token and the transaction would still be executed.

To execute an attack to capture the maximum possible slippage of 0.0909 a searcher would first determine what transaction they need to execute such that the supply of A and B tokens in the pool when the user’s transaction is executed results in the user receiving 0.8181 B tokens. Then after the user’s transaction, the searcher executes another transaction in the opposite direction that leaves the supply in the pools where it would have been without the attack, netting them the slippage at the user’s expense.

$$
\begin{align*}
Let A_1 &= \text{the supply of token A after the attacker's initial transaction} \\
Let B_1 &= \text{the supply of token B after the attacker's initial transaction} \\
\end{align*}
$$

and in order for $A * B = 100$ to be true,

$$
\begin{align*}
(A_1 + 1.0000) * (B_1 - 0.8181) &= 100 \\
(A_1 + 1.0000) * (\frac{100}{A_1} - 0.8181) &= 100 \\
A_1 = 10.5667
\end{align*}
$$

which means the swap the searcher executes initially is for 0.5363 B tokens in exchange for 0.5667 A tokens. After the user's transaction the searcher swaps their 0.5667 A tokens for 0.4454 B tokens to net 0.0909 B tokens from the sandwich attack.

|        | A         | B       | k / price    |
| ------ | -------: | -------: |------------: |
| $t_0$  | 10.0000  | 10.0000  |    100.0000  |
| $pre$  |  0.5667  | -0.5363  |      0.9463  |
| $t_1$  | 10.5667  |  9.4637  |    100.0000  |
| $user$ |  1.0000  | -0.8182  |     -0.8182  |
| $t_2$  | 11.5667  |  8.6455  |    100.0000  |
| $post$ | -0.5667  |  0.4454  |      0.7860  |
| $t_3$  | 11.0000  |  9.0909  |    100.0000  |

### Examples

In [5]:
if DB_AVAIL:
    df_swap = pd.read_sql("SELECT * from swaps where block_number = 13998686", conn)
    df_swap.to_csv("data/swap.csv", index=False)
else:
    df_swap = pd.read_csv("data/swap.csv")

In [ ]:
df_swap

,created_at,abi_name,transaction_hash,block_number,protocol,contract_address,from_address,to_address,token_in_address,token_in_amount,token_out_address,token_out_amount,trace_address,error,transaction_position
0,2022-01-13 18:11:48.783264,UniswapV2Pair,0x405e13ce97af14d62f6ccf177fe6216543b0e0c56aab...,13998686.0,None,0xdb4f0e1f537ed1d4bc4714367dec1e2417f35592,0x1111111254fb6c44bac0bed2854e76f90643097d,0x2ba1d521595c5b31b177fd5263f8b002f570e92a,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,3.000000e+16,0x3b484b82567a09e2588a13d54d032153f0c0aee0,3.074319e+25,[3],None,133.0
1,2022-01-13 18:11:48.783264,UniswapV3Pool,0x54260ca010ed0d1e7426de6aeff325aa23dcbb8c6260...,13998686.0,None,0x3416cf6c708da44db2624d63ea0aaef7113527c6,0xfa12612a0c2ef2e88861794422d379f1c92f70b4,0xfa12612a0c2ef2e88861794422d379f1c92f70b4,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,4.403631e+09,0xdac17f958d2ee523a2206206994597c13d831ec7,4.402393e+09,[0],None,30.0
2,2022-01-13 18:11:48.783264,UniswapV3Pool,0x59b03593f8443634bf23235154fb2e2d049e9f843208...,13998686.0,None,0x4b5ab61593a2401b1075b90c04cbcdd3f87ce011,0x68b3465833fb72a70ecdf485e0e4c7bd8665fc45,0x661000211d2bedb5ef54c7cc769178ad4a5a307a,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,1.599790e+17,0xf4d2888d29d722226fafa5d9b24f9164c092421e,1.250000e+20,"[0, 0]",None,91.0
3,2022-01-13 18:11:48.783264,UniswapV2Pair,0xa14eb4628ae728b2d4e47a4ea70694c1854351389cc2...,13998686.0,None,0xf3fd3aceb26581ccefb502825239f072933a25b3,0x1111111254fb6c44bac0bed2854e76f90643097d,0x862ce7605f8903369211ec63d446b64ef8fba18c,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,1.000000e+15,0x0f4c00139602ab502bc7c1c0e71d6cb72a9fb0e7,3.541058e+18,[3],None,128.0
4,2022-01-13 18:11:48.783264,UniswapV3Pool,0xa2a2339d6abe5c2f4411c5c4641ddd10934bac363dd4...,13998686.0,None,0x82743c07bf3be4d55876f87bca6cce5f84429bd0,0xdef1c0ded9bec7f1a1670819833240f027b25eff,0x74de5d4fcbf63e00296fd95d33236b9794016631,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,3.965000e+17,0x2b591e99afe9f32eaa6214f7b7629768c40eeb39,5.621688e+11,"[0, 0, 0, 0, 1]",None,82.0
5,2022-01-13 18:11:48.783264,BPool,0xe8f6a308aabb4ed33847374d356e42caaf190aed5afb...,13998686.0,balancer_v1,0xc697051d1c6296c24ae3bcef39aca743861d9a81,0xa57bd00134b2850b2a1c55860c9e9ea100fdd6cf,0xa57bd00134b2850b2a1c55860c9e9ea100fdd6cf,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,6.317180e+19,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,9.653321e+20,"[0, 18]",None,4.0
